# COVID-19 Analysis - Argentina - 05/09/2020. (R Programming)

### Import the necessary libraries for the analysis.

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) 
library(forcats)
library(repr)
library(caret)
library(ROCR)
library(grid)
library(CatEncoders)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input/covidargentina/Covid19Casos.csv")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import the dataset.

In [ ]:
covid <-  as.data.frame(read.csv('../input/covidargentina/Covid19Casos.csv',stringsAsFactors = FALSE, encoding = 'UTF-8'))

## Part 1 - Know the data

### Let's take a look at the dataset.

In [ ]:
# We are going to have a general overview about the data involved.
glimpse(covid)

In [ ]:
# Let's see the provinces of Argentina and test types.
unique(covid$residencia_provincia_nombre) 
unique(covid$clasificacion_resumen)

### Now, let's modify a little bit the data. 

In [ ]:
# We are going to translate some values to English...
covid$clasificacion_resumen[covid$clasificacion_resumen == 'Descartado'] <- 'Negative'
covid$clasificacion_resumen[covid$clasificacion_resumen == 'Sospechoso'] <- 'Suspicious'
covid$clasificacion_resumen[covid$clasificacion_resumen == 'Confirmado'] <- 'Confirmed'
covid$clasificacion_resumen[covid$clasificacion_resumen == 'Sin Clasificar'] <- 'Unclassified'

In [ ]:
#... and change the dates format.
covid$fecha_apertura <- as.Date(covid$fecha_apertura)
covid$fecha_diagnostico <- as.Date(covid$fecha_diagnostico)
covid$fecha_inicio_sintomas <- as.Date(covid$fecha_inicio_sintomas, format = "%Y-%m-%d")
covid$ultima_actualizacion <- as.Date(covid$ultima_actualizacion, format = "%Y-%m-%d")

### 1.1 - Quantity of tests per province.

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

ggplot(covid,aes(x = reorder(residencia_provincia_nombre,residencia_provincia_nombre,function(x)-length(x)),
                             fill=factor(clasificacion_resumen))) +
  geom_bar() +
  scale_fill_brewer(palette = "RdGy")+
  scale_y_continuous(labels = scales::comma, )+
  coord_flip() +
  ggtitle("Testing per Province")+
  xlab("Province") +
  ylab("Tests Quantity") +
  labs(fill = "Status") +
  theme(legend.background = element_rect(fill = "lightblue",colour = "grey50",size = 1))
       


### 1.2 - Quantity of confirmed cases.

In [ ]:
confirmed <- as_tibble(covid %>%
          filter(covid["clasificacion_resumen"] == "Confirmed") %>%
          count(residencia_provincia_nombre, sort = TRUE ))%>%
          rename(Quantity_of_cases = n)

In [ ]:
head(confirmed)

In [ ]:
options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

ggplot(confirmed, aes(y= Quantity_of_cases, x = reorder(residencia_provincia_nombre, desc(Quantity_of_cases)))) +
 geom_col(fill='blue') +
 scale_y_continuous(labels = scales::comma)+
 coord_flip() +
 ggtitle("Confirmed cases per Province")+
 xlab("Province") +
 ylab("Confirmed Cases Quantity")


### 1.3 - Evolution of cases through time.

In [ ]:
# Now we have to take a look if there are any missing information and format the dates.
sum(is.na(covid$fecha_apertura))
sum(is.na(covid$clasificacion_resumen))

In [ ]:
# Let's analyze the "NA" value.
na_value <- covid %>% filter(is.na(covid$fecha_apertura) == TRUE)
na_value

In [ ]:
# It seems that the value does not follow the structure of the rest of the dataset.
# So, we can proceed with the plotting of the information. 

options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

ggplot(covid, aes(x = fecha_apertura ,fill = clasificacion_resumen ))+
  geom_bar() +
  scale_x_date(date_labels = "%m-%Y", date_break = "1 month" )+
  scale_y_continuous(labels = scales::comma)+
  ggtitle("Tests through time")+
  xlab("Month") +
  ylab("Quantity of tests")


### 1.3.1 - Evolution of cases through time (without suspicious cases).

In [ ]:
# We are going to generate another dataset without the "Suspicious" cases
non_suspicious <- covid %>% filter(clasificacion_resumen != 'Suspicious')

In [ ]:
# And let's plot it. 

options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

ggplot(non_suspicious, aes(x = fecha_apertura ,fill = clasificacion_resumen))+
  geom_density(alpha = 0.25) +
  facet_grid(clasificacion_resumen ~.)+
  scale_x_date(date_labels = "%m-%Y", date_break = "1 month" )+
  scale_y_continuous(breaks = c(0.01, 0.02), labels = c("10,000", "20,000"))+
  ggtitle("Confirmed cases through time")+
  xlab("Month") +
  ylab("Quantity of cases")

### 1.4 - Quantity of deaths through time.

In [ ]:
# Generate a new dataset only with deaths
death <- covid %>% filter(fallecido == "SI")

# And plot it.
options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

ggplot(death, aes(x = fecha_apertura ,fill = fallecido))+
  geom_density(alpha = 0.75) +
  scale_x_date(date_labels = "%m-%Y", date_break = "1 month" )+
  scale_y_continuous(breaks = c(0.0025, 0.005,0.0075,0.01,0.0125), labels = c("2,500","5,000","7,500","10,000","12,500"))+
  ggtitle("Deaths through time")+
  xlab("Month") +
  ylab("Quantity of cases")

### 1.5 - Evolution of ventilators needed through time.

In [ ]:
# New dataset only with the required ventilators
ventilator <- covid %>% filter(asistencia_respiratoria_mecanica == "SI")

# Let's see the graph.
options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

ggplot(ventilator, aes(x = fecha_apertura ,fill = asistencia_respiratoria_mecanica))+
  geom_density(alpha = 0.75) +
  scale_x_date(date_labels = "%m-%Y", date_break = "1 month" )+
  scale_y_continuous(breaks = c(0.002, 0.004,0.006,0.008), labels = c("2,000","4,000","6,000","8,000"))+
  ggtitle("Ventilators needed through time")+
  xlab("Month") +
  ylab("Quantity of ventilators needed") +
  scale_fill_brewer(palette = "BuGn")

## Part 2 - Data Preparation

### Now we are going to prepare the data to run a logistic regression model in order to predict if people with certain characteristics will survive COVID or if they will not. 

### 2.1 - Creating a new dataset only with useful information.

In [ ]:
# First of all, let's select only the relevant elements for the model and creat a new dataframe.
covid_analisis <- covid %>% select('fallecido',
                                   'sexo',
                                   'origen_financiamiento',
                                   'asistencia_respiratoria_mecanica',
                                   'edad') 
glimpse(covid_analisis)


In [ ]:
# And let's change the characters values of our target for numbers.
covid_analisis$fallecido[covid_analisis$fallecido == 'NO'] <- 0
covid_analisis$fallecido[covid_analisis$fallecido == 'SI'] <- 1

### 2.2 - Taking a look at the NA's.

In [ ]:
# We have to analyze if there are NA's in our dataframe.
colSums(is.na(covid_analisis))

### We have 1541 NA's in the "edad" field, as it does not represent a lot of cases compared with the rows of the entire dataframe, we will replace the NA's with the average. Otherwise, it will affect the model. 

In [ ]:
# Replace the NA's with the average.
for(i in 1:ncol(covid_analisis)){
  covid_analisis[is.na(covid_analisis[,i]), i] <- mean(covid_analisis[,i], na.rm = TRUE)
}

In [ ]:
# Let's define all categorical values as factors and the age as interger.
covid_analisis$fallecido  <- as.factor(covid_analisis$fallecido)
covid_analisis$sexo <- as.factor(covid_analisis$sexo)
covid_analisis$origen_financiamiento <- as.factor(covid_analisis$origen_financiamiento)
covid_analisis$asistencia_respiratoria_mecanica  <- as.factor(covid_analisis$asistencia_respiratoria_mecanica)
covid_analisis$edad  <- as.integer(covid_analisis$edad)

### 2.3 - Including the target in the "x" group and the variables in the "y" group. 

In [ ]:
x <- 'fallecido'
y <- c('sexo',
       'residencia_provincia_nombre',
       'origen_financiamiento',
       'asistencia_respiratoria_mecanica',
       'edad')

### 2.4 - Splitting the datasets into train and test. 

In [ ]:
# set seed for reproducibility
set.seed(123)

# making a train index
train_index <- sample(c(TRUE, FALSE), replace = TRUE, size = nrow(covid_analisis), prob = c(0.2, 0.8))

# split the data according to the train index
train <- as.data.frame(covid_analisis[train_index, ])
test <- as.data.frame(covid_analisis[!train_index, ])

## Part 3 - Modelling

### 3.1 - Train the model.

In [ ]:
model <- glm(fallecido ~ . , data = train, family = binomial(link = 'logit'))

In [ ]:
#Let's take a look at the predictions.
options(repr.plot.width=15, repr.plot.height=10)
theme_set(theme_classic(base_size = 25))

# make predictions on the test set
train$pred <- predict(model, newdata=train, type="response")
test$pred <- predict(model, newdata=test, type="response")

# plot histogram of predictions
data.frame(preds <- train$pred ) %>%
    ggplot(aes(x = preds)) + 
    geom_histogram(bins = 50, fill = 'grey50') +
    labs(title = 'Histogram of Predictions') 

# print range of predictions
print(round(range(train$pred),2))

# print median of predictions
print(median(train$pred))


### 3.2 - Exploring trade-off between precission and recall. 

In [ ]:
predObj <- prediction(train$pred, train$fallecido)
precObj <- performance(predObj, measure="prec")
recObj <- performance(predObj, measure="rec")

In [ ]:
precision <- (precObj@y.values)[[1]]
prec.x <- (precObj@x.values)[[1]]
recall <- (recObj@y.values)[[1]]

In [ ]:
rocFrame <- data.frame(threshold=prec.x, precision=precision,recall=recall)
nplot <- function(plist) {
                           n <- length(plist)
                           grid.newpage()
                           pushViewport(viewport(layout=grid.layout(n,1)))
                           vplayout=function(x,y) {viewport(layout.pos.row=x, layout.pos.col=y)}
                           for(i in 1:n) {
                           print(plist[[i]], vp=vplayout(i,1))
                                         }
                          }
pnull <-mean(as.numeric(train$fallecido))

p1 <- ggplot(rocFrame, aes(x=threshold)) +
      geom_line(aes(y=precision/pnull)) +
      coord_cartesian(xlim = c(0,0.05), ylim=c(0,0.25) ) +
      geom_vline(xintercept = 0.02, color="red", linetype = 2)
p2 <- ggplot(rocFrame, aes(x=threshold)) +
      geom_line(aes(y=recall)) +
      coord_cartesian(xlim = c(0,0.05) ) +
      geom_vline(xintercept = 0.02, color="red", linetype = 2)

nplot(list(p1, p2))

Once we see the charts, we can conclude that 0.02 might be a good trade off. 

### 3.3 - Confusion matrix.
[(Click for further information about this matrix)](https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62)

In [ ]:
ctab.test <- table(pred=test$pred>0.02, fallecido =test$fallecido)
ctab.test

### 3.3.4 - Precesion, recall and accuracy

![](https://miro.medium.com/max/2400/1*uR09zTlPgIj5PvMYJZScVg.png)

In [ ]:
precision <- ctab.test[2,2]/sum(ctab.test[2,])
paste("precision = ",precision)

recall <- ctab.test[2,2]/sum(ctab.test[,2])
paste("recall = ",recall)

accuracy <- (ctab.test[2,2]+ctab.test[1,1])/sum(ctab.test)
paste("accuracy = ",accuracy)

### 3.4 - ROC Curve. 

![](http://)[(Click here for further information)](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5)

In [ ]:
preds <- predict(model, newdata = test, type = "response")

roc_data <- data.frame(
    p0.3 = ifelse(preds > 0.3, 1, 0),
    p0.2 = ifelse(preds > 0.2, 1, 0),
    p0.1 = ifelse(preds > 0.1, 1, 0),
    p0.05 = ifelse(preds > 0.05, 1, 0),
    p0.04 = ifelse(preds > 0.04, 1, 0),
    p0.03 = ifelse(preds > 0.03, 1, 0),
    p0.02 = ifelse(preds > 0.02, 1, 0),
    p0.01 = ifelse(preds > 0.01, 1, 0))

In [ ]:
# true positive (hit) rate
tpr <- function(pred, actual) {
    res <- data.frame(pred, actual)
    sum(res$actual == 1 & res$pred == 1) / sum(actual == 1)
}

# false positive rate
fpr <- function(pred, actual) {
    res <- data.frame(pred, actual)
    sum(res$actual == 0 & res$pred == 1) / sum(actual == 0)
}

In [ ]:
actual <- test$fallecido

In [ ]:
# reshape to long format and get fpr and tpr for each threshold
roc_data <- roc_data %>% 
    gather(key = 'threshold', value = 'pred') %>% 
    group_by(threshold) %>%
    summarize(tpr = tpr(pred, actual = actual), 
              fpr = fpr(pred, actual = actual))

In [ ]:
# set x and y tick marks
breaks <-  c(0, 0.2, 0.4, 0.6, 0.8, 1)

# get labels for plotting break points
labels <- substr(roc_data$threshold, start = 2, stop = 5)

# plot the ROC curve
ggplot(data = roc_data, aes(x = fpr, y = tpr)) + 
    geom_line() + 
    geom_text(aes(label = labels), nudge_x = 0.05) + 
    geom_abline(intercept = 0, slope = 1, linetype = 'dashed') + 
    scale_x_continuous(limits = c(0, 1), breaks = breaks) + 
    scale_y_continuous(limits = c(0,1), breaks = breaks) + 
    labs(x = 'False Positive Rate', y = 'True Positive Rate', title = 'ROC Curve') 

### 3.5 - Model Summary. 

In [ ]:
summary(model)